In [1]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

from pyomo.core.util import quicksum

import pandas as pd
from datetime import date, datetime
import itertools
import numpy as np

from bank_schedule import data as data_
from bank_schedule import cluster


In [2]:
data = data_.Data('/Users/sykuznetsov/Documents/GitHub/bank_schedule/data/raw')

In [3]:
clustered = cluster.clusterize_atm(data, allowed_percent=0.01, n_clusters=20)

In [4]:
clustered[clustered["label"] == 3]

,TID,label
623,699560,3
624,657216,3
625,687620,3
626,628631,3
627,687665,3
...,...,...
1498,619583,3
1512,635229,3
1540,689146,3
1541,644825,3


In [5]:
money_in = data.get_money_in()

tids_chosen = clustered[clustered["label"] == 3]['TID'].unique()

distance_matrix = data.get_distance_matrix()

money_in = money_in[money_in['TID'].isin(tids_chosen)]

distance_matrix_filtered = distance_matrix[(distance_matrix["Origin_tid"].isin(tids_chosen)) & (distance_matrix["Destination_tid"].isin(tids_chosen))]

money_in = money_in[(money_in["date"].dt.date >= date(2022, 11, 1)) & (money_in["date"].dt.date <= date(2022, 11, 14))]

date_num_dict = {date: num for num, date in enumerate(sorted(money_in['date'].unique()))}

money_in['date'] = money_in['date'].map(date_num_dict)

params = data.get_params_dict()

quantity_cars = 6

kwargs = {
    'ratioGap': 0.0001, 
    'sec': 300,
    'threads': 10}

M = 100000000000

# technical_cost = 0.0000001

technical_cost = 1


money_in_dict = money_in.set_index(['TID', 'date']).to_dict()['money_in']

In [6]:

# BIG_NUM = 40*60
# postomat_places = list(fixed_points) + list(possible_postomats)
# center_mass_set = set(distances['id_center_mass'].to_list())
# metro_points_set = set(distanses_metro['object_id_metro'].to_list())

# distances_dict = {(id_center_mass, postomat_place_id): walk_time for _, postomat_place_id, id_center_mass , _, _, walk_time in distances.itertuples()}
# distances_metro_dict = {(object_id_metro, object_id): walk_time for _, object_id, object_id_metro ,  _, walk_time in distanses_metro.itertuples()}



# Создание конкретной модели pyomo
model = pyo.ConcreteModel()

model.TIDS = pyo.Set(initialize=money_in['TID'].unique())

model.DATES = pyo.Set(initialize=money_in['date'].unique())

model.MAX_MONEY = params['max_money']

model.MAX_DAYS_INC = params['max_days_inc']

model.OVERNIGHT_BY_DAY = params['overnight']/100/365

model.COST_INC_PERS = params['cost_inc_pers']

model.COST_INC_MIN = params['cost_inc_min']

model.MIN_WAIT = params["min_wait"]

model.TIDS_wuth_null = pyo.Set(initialize=np.append(money_in['TID'].unique(), -1))

model.MINUTES_CARS = (datetime.combine(date.today(), params['day_end'] ) - \
                      datetime.combine(date.today(), params['day_start']))\
                      .total_seconds() / 60

model.distance_matrix_dict = distance_matrix_filtered.set_index(["Origin_tid",	"Destination_tid"]).to_dict()['Total_Time']

model.days_from_inc_dict = {tid: 0 for tid in model.TIDS}

model.money_in_dict = money_in_dict

model.money_start_dict = {row[1]['TID']: row[1]['money'] for row in data.get_money_start().iterrows()}

model.MAX_DATE = max(model.DATES)


# Переменные
model.money_inc = pyo.Var(model.TIDS, model.DATES, within=pyo.Binary, initialize=0)


model.money_inside_TID = pyo.Var(model.TIDS, model.DATES,  within=pyo.NonNegativeReals, initialize=0 )

def con_money_inside_TID_1(model, tid, date):
    if date == 0:
        # TODO Проверить не теряем ли мы день
        out = model.money_start_dict[tid]
    else:
        out = model.money_inside_TID[tid, date - 1] + model.money_in_dict[(tid, date - 1)]
    return  out -  model.money_inc[tid, date] * M <= model.money_inside_TID[tid, date]
model.con_money_inside_TID_1 = pyo.Constraint(model.TIDS, model.DATES, rule=con_money_inside_TID_1)



def con_money_inside_TID_2(model, tid, date):
    return model.money_inside_TID[tid, date] <= (1 - model.money_inc[tid, date]) * M
model.con_money_inside_TID_2 = pyo.Constraint(model.TIDS, model.DATES, rule=con_money_inside_TID_2)


def con_max_money(model, tid, date):
    return model.money_inside_TID[tid, date] <= model.MAX_MONEY
model.con_max_money = pyo.Constraint(model.TIDS, model.DATES, rule=con_max_money)



def con_max_inc(model, date):
    return quicksum([model.money_inc[tid, date] for tid in model.TIDS]) <= 40
model.con_max_inc = pyo.Constraint(model.DATES, rule=con_max_inc)




# model.days_from_inc = pyo.Var(model.TIDS, model.DATES,  within=pyo.NonNegativeIntegers, initialize=0 )

# # Можно не водить переменные, а просто ограничиться ограничением
# def con_days_from_inc(model, tid, date):

#     if date == 0:
#         out = model.days_from_inc_dict[tid]
#     else:
#         out = model.days_from_inc[tid, date - 1] + 1


#     return out -  model.money_inc[tid, date] * M <= model.days_from_inc[tid, date]



# model.con_days_from_inc = pyo.Constraint(model.TIDS, model.DATES, rule=con_days_from_inc)



def con_days_from_inc(model, tid, date):

    right_date_border_raw = date + model.MAX_DAYS_INC - max(model.days_from_inc_dict[tid] - date, 0)
    right_date_border = min(right_date_border_raw, model.MAX_DATE) + 1
    return quicksum([ model.money_inc[tid, date_iter] for date_iter in  range(date, right_date_border)]) >= 1 + min(0, (model.MAX_DATE - date) - model.MAX_DAYS_INC )

model.con_days_from_inc = pyo.Constraint(model.TIDS, model.DATES, rule=con_days_from_inc)


# def con_max_days_inc(model, tid, date):
#     return model.days_from_inc[tid, date] <= model.MAX_DAYS_INC
# model.con_max_days_inc = pyo.Constraint(model.TIDS, model.DATES, rule=con_max_days_inc)



def costs_from_money(model, tid, date):
    return model.money_inside_TID[tid, date] * model.OVERNIGHT_BY_DAY
model.costs_from_money = pyo.Expression(model.TIDS, model.DATES, rule=costs_from_money )


# Затраты на инкассацию
# model.costs_for_inc = pyo.Var(model.TIDS, model.DATES,  within=pyo.NonNegativeReals, initialize=0 )
# def con_costs_for_inc_1(model, tid, date):
#     return model.money_inc[tid, date] * model.COST_INC_MIN <= model.costs_for_inc[tid, date]
# model.con_costs_for_inc_1 = pyo.Constraint(model.TIDS, model.DATES, rule=con_costs_for_inc_1 )

# def con_costs_for_inc_2(model, tid, date):
#     return model.money_inside_TID[tid, date] * model.COST_INC_PERS -  model.money_inc[tid, date] * M <= model.costs_for_inc[tid, date]
# model.con_costs_for_inc_2 = pyo.Constraint(model.TIDS, model.DATES, rule=con_costs_for_inc_2 )

def costs_for_inc(model, tid, date):
    return model.money_inc[tid, date] * model.COST_INC_MIN
model.costs_for_inc = pyo.Expression(model.TIDS, model.DATES, rule=costs_for_inc )


#Последовательность маршрута
model.route = pyo.Var(model.TIDS_wuth_null, model.TIDS_wuth_null, model.DATES, within=pyo.Binary, initialize=0 )

def con_route_1(model, tid, date):
    return quicksum([model.route[tid, tid2, date] for tid2 in model.TIDS_wuth_null if tid != tid2]) == model.money_inc[tid, date]
model.con_route_1 = pyo.Constraint(model.TIDS, model.DATES, rule=con_route_1)

def con_route_2(model, tid, date):
    return quicksum([model.route[tid2, tid, date] for tid2 in model.TIDS_wuth_null if tid != tid2]) == model.money_inc[tid, date]
model.con_route_2 = pyo.Constraint(model.TIDS, model.DATES, rule=con_route_2 )

def con_route_3(model, date):
    return quicksum([model.route[tid2, -1, date] for tid2 in model.TIDS]) == 1
model.con_route_3 = pyo.Constraint(model.DATES, rule=con_route_3 )


def con_route_4(model, date):
    return quicksum([model.route[-1, tid2, date] for tid2 in model.TIDS]) == 1
model.con_route_4 = pyo.Constraint(model.DATES, rule=con_route_4 )


# def con_route_2(model, tid1, tid2, date):
#     return 1 - (model.money_inc[tid1, date] + model.money_inc[tid2, date]) * M <= model.route[tid1, tid2, date]
# model.con_route_2 = pyo.Constraint(TIDS, TIDS, DATES, rule=con_route_2 )




def con_max_time(model, date):
    return quicksum([model.route[tid1, tid2, date] * model.distance_matrix_dict[(tid1, tid2)] for tid1, tid2 in itertools.product(list(model.TIDS), list(model.TIDS)) if tid1 != tid2]) +\
    quicksum([model.money_inc[tid, date] * model.MIN_WAIT for tid in model.TIDS])\
    <= model.MINUTES_CARS
model.con_max_time = pyo.Constraint(model.DATES, rule=con_max_time )




model.OBJ = pyo.Objective(expr=
                                # quicksum([model.money_inside_TID[tid, date] for tid, date in itertools.product(list(model.TIDS), list(model.DATES)) ]) +
                                # quicksum([model.days_from_inc[tid, date] for tid, date in itertools.product(list(model.TIDS), list(model.DATES)) ]) +
                                (1/technical_cost) * quicksum([model.costs_from_money[tid, date] for tid, date in itertools.product(list(model.TIDS), list(model.DATES)) ]) +
                                (1/technical_cost) * quicksum([model.costs_for_inc[tid, date] for tid, date in itertools.product(list(model.TIDS), list(model.DATES)) ]), sense=pyo.minimize)
# minimize

# , executable="/usr/local/Cellar/cbc/2.10.8/bin/cbc"
opt = SolverFactory('cbc')

for key in kwargs:
    opt.options[key] = kwargs[key]

results = opt.solve(model)

print(results['Problem'])
print(results['Solver'])
print(model.money_inside_TID.extract_values())
print(model.costs_from_money.extract_values())
# print(model.days_from_inc.extract_values())
print(model.costs_for_inc.extract_values())



solution_dict = model.money_inc.extract_values()
solution_pd = pd.DataFrame(solution_dict.items(), columns=['index', 'inc'])
solution_pd['TID'], solution_pd['date_num'] = zip(*solution_pd['index'])

display(solution_pd)

In [11]:
solution_pd.groupby('date_num').sum('sum')

,inc,TID
date_num,,
0,3.0,18410828
1,7.0,18410828
2,6.0,18410828
3,1.0,18410828
4,3.0,18410828
5,5.0,18410828
6,5.0,18410828
7,4.0,18410828
8,1.0,18410828


In [26]:
list(range(4,4))

[]

In [ ]:
def con_days_from_inc(model, tid, date):

    if date == 0:
        out = model.days_from_inc_dict[tid]
    else:
        out = model.days_from_inc[tid, date - 1] + 1


    return quicksum([model.costs_from_money[tid, date] for tid, date in itertools.product(list(model.TIDS), list(model.DATES)) ])

In [ ]:

    return solution_pd.loc[solution_pd['place_postomat'] > 0, 'object_id'].to_list()

# BIG_NUM = 40*60
# postomat_places = list(fixed_points) + list(possible_postomats)
# center_mass_set = set(distances['id_center_mass'].to_list())
# metro_points_set = set(distanses_metro['object_id_metro'].to_list())

# distances_dict = {(id_center_mass, postomat_place_id): walk_time for _, postomat_place_id, id_center_mass , _, _, walk_time in distances.itertuples()}
# distances_metro_dict = {(object_id_metro, object_id): walk_time for _, object_id, object_id_metro ,  _, walk_time in distanses_metro.itertuples()}

# Создание конкретной модели pyomo
model = pyo.ConcreteModel()

# Переменные
model.has_postomat = pyo.Var(postomat_places, within=pyo.Binary, initialize=0)

if precalculated_points is not None:
    for point in precalculated_points:
        model.has_postomat[point] = 1


for fixed_point in fixed_points:
    model.has_postomat[fixed_point].fix(1)

model.center_mass_time_to_nearest_postamat = pyo.Var(population_points, within=pyo.NonNegativeReals)

#Ограничения

def con_center_mass_time_to_nearest_postamat(model, *data):
    _, id_center_mass, postomat_place_id = data
    return model.center_mass_time_to_nearest_postamat[id_center_mass] >= distances_dict[(id_center_mass, postomat_place_id)] * model.has_postomat[postomat_place_id]

model.con_center_mass_time_to_nearest_postamat = pyo.Constraint( list(distances[['id_center_mass',	'object_id']].itertuples()) ,rule=con_center_mass_time_to_nearest_postamat)

def center_mass_has_postomat(model, center_mass_id):
    only_needed_dist = distances.loc[distances['id_center_mass'] == center_mass_id]
    out = 0
    for object_id in only_needed_dist['object_id']:
        out += model.has_postomat[object_id]
    return out


model.center_mass_has_postomat = pyo.Expression(population_points, rule=center_mass_has_postomat )



def con_center_mass_has_postomat(model, center_mass_id):
    return model.center_mass_time_to_nearest_postamat[center_mass_id] >= (1 - model.center_mass_has_postomat[center_mass_id]) * BIG_NUM 


model.con_center_mass_has_postomat = pyo.Constraint(population_points, rule=con_center_mass_has_postomat)



model.metro_time_to_nearest_postamat = pyo.Var(object_id_metro_list, within=pyo.NonNegativeReals)

def con_metro_time_to_nearest_postamat(model, *data):
    _, object_id_metro, postomat_place_id = data
    return model.metro_time_to_nearest_postamat[object_id_metro] >= distances_metro_dict[(object_id_metro, postomat_place_id)] * model.has_postomat[postomat_place_id]

model.con_metro_time_to_nearest_postamat = pyo.Constraint( list(distanses_metro[['object_id_metro',	'object_id']].itertuples()) ,rule=con_metro_time_to_nearest_postamat)


def metro_has_postomat(model, metro_id):
    only_needed_dist = distanses_metro.loc[distanses_metro['object_id_metro'] == metro_id]
    out = 0
    for object_id in only_needed_dist['object_id']:
        out += model.has_postomat[object_id]
    return out


model.metro_has_postomat = pyo.Expression(object_id_metro_list, rule=metro_has_postomat )

def con_metro_has_postomat(model, metro_id):
    return model.metro_time_to_nearest_postamat[metro_id] >= (1 - model.metro_has_postomat[metro_id]) * BIG_NUM 


model.con_metro_has_postomat = pyo.Constraint(object_id_metro_list, rule=con_metro_has_postomat)


model.needed_postamats = pyo.Constraint(expr=sum([model.has_postomat[p] for  p in postomat_places]) <= quantity_postamats_to_place)

sum_center_mass = sum(model.center_mass_time_to_nearest_postamat[p] * population_dict[p] for p in population_points) 
sum_metro = sum(model.metro_time_to_nearest_postamat[p] * population_dict[p] for p in object_id_metro_list)
# # Целевая
model.OBJ = pyo.Objective(expr=((1 - metro_weight) * sum_center_mass + (metro_weight) * sum_metro), sense=pyo.minimize)
# minimize

# , executable="/usr/local/Cellar/cbc/2.10.8/bin/cbc"
opt = SolverFactory('cbc')

for key in kwargs:
    opt.options[key] = kwargs[key]

results = opt.solve(model)


optimised_list = get_chosen_postomats(model)

optimised_list_no_fixed = list(set(optimised_list).difference(set(fixed_points)))